# Scala's Collection Library

### References

A gentle guide to the Scala's collection library can be found online in the Scala site:

https://docs.scala-lang.org/overviews/collections-2.13/introduction.html

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import _root_.org.scalatest._

## Collection types

Collection types can be classified along two major dimensions:
- Mutable/Immutable: mutable collections can be modified in-place; immutable collections return a new instance when they are updated.
- Sequences/Sets/Maps. These are ordered/unordered collections and key-value collections, respectively. 

There are many different implementations of sequences, sets and maps, as summarized by these figures (cf. Scala's [guide](https://docs.scala-lang.org/overviews/collections-2.13/overview.html))

![](images/collections-hierarchy.svg)

![](images/collections-hierarchy-inmutable.svg)

![](images/collections-mutable.svg)

How do we choose from this huge number of collection types and implementations? The colletion type is chosen according to the functionality that we demand from our collection, e.g., should it be _ordered_?, can there be repeated elements?, are elements associated with keys?, etc. In order to choose the right implementation of the collection type (e.g. should I use a `Vector` or a `List`?) we must take into account the complexity of the corresponding implementation: 

https://docs.scala-lang.org/overviews/collections-2.13/performance-characteristics.html

From now on, we will work with immutable collection types only.

## `Map`s of key-value pairs

`Map`s are key-value collections (i.e. they are like sets of key-value pairs indexed by their keys).

In [ ]:
val m: Map[String, Int] = 
    Map(("a", 5), ("b", 10), ("c", 11))

Specially for key-value pairs, we commonly write `Tuple2` values with the following syntax: 

In [ ]:
val m: Map[String, Int] = 
    Map("a" -> 5, "b" -> 10, "c" -> 11)

Common operations on maps: 

In [ ]:
// retrieving values of existing and non-existing keys

m("a"): Int
m.get("a"): Option[Int]
try{ m("d") } catch { case _ => "exception raised" }
m.get("d") 

In [ ]:
// retrieving all keys and values

m.keys: Iterable[String]
m.values: Iterable[Int]

The default implementation types of these `Iterable`s are `collection.immutable.Set` and the general `View` type. Views are _lazy_ collections which offer better performance. For more information on views consult the Scala [guide](https://docs.scala-lang.org/overviews/collections-2.13/views.html). The only thing that we need from views and the general `Iterable` collection type is that they can be converted to a concrete collection type with conversors `toList`, `toSet`, `toMap`, etc. 

In [ ]:
// Converting to proper implementation types
m.keys.toList
m.keys.toSet
m.values.toList
m.values.toSet

In [ ]:
// Mapping values (toMap is required to convert the view type `MapView` to `Map`)

val m2: Map[String, Boolean] = 
    m.mapValues((value: Int) => value % 2 == 0).toMap

In [ ]:
// Mapping whole entries, not only values

m.map((entry: (String, Int)) => entry._2 % 2 == 0)
m.map{ case (key: String, value: Int) => value % 2 == 0 }

In [ ]:
// Filter also return a general iterable (a list, by default)
m.filter{ case (key: String, value: Int) => value > 10 }

We can obtain a `Map` from a list of pairs with `toMap`:

In [ ]:
val l: List[(Int, String)] =  
    List((1, "a"), (2, "b"), (1, "c"), (2, "d"), (3, "a"))

l.toMap

Note that only one value is kept for a single key. If we want all values, we can use `groupBy`:

In [ ]:
l.groupBy(entry => entry._1)

# Set collections

`Set`s are unordered collections of unique elements. The following two sets are equal: 

In [ ]:
Set(1,2,2,3) == Set(3,1,2)
// compare with lists:
List(1,2,2,3) == List(3,1,2)

Common operations on sets: 

In [ ]:
// Filtering elements
Set(-1, 0, 1, 3, -5, 2, 4).filter(e => e > 0)

In [ ]:
// Mapping elements
Set(-1,-4,-3,5).map(e => e.abs)

In [ ]:
// Flatmapping elements
Set(1,2,3).flatMap(e => Set(e,-e))

In [ ]:
// Common set operations
Set(1,2,3).subsetOf(Set(1,2,3,4))
Set(1,2,3) subsetOf Set(1,2)
Set(1,2,3) diff Set(1,2)
Set(1) diff Set(3,4)
Set(1,2,3) union Set(1,2,3,4,5)

## Implementing & querying data models

The Scala collections greatly facilitate the implementation and querying of data models. For instance, the following classes model the structure of an organization which consists of departments, employees and tasks that employees can perform. 

In [ ]:
// departments

case class Department(id: Department.Id)
object Department{
    type Id = String
}

// tasks 

case class Task(id: Task.Id, hours: Int)
object Task{
    type Id = String
}

// Employees

case class Employee(id: Employee.Id, dpt: Department.Id)
object Employee{
    type Id = String
}

// The whole organization

case class Organization(
    departments: Map[Department.Id, Department], 
    tasks: Map[Task.Id, Task],
    employees: Map[Employee.Id, Employee], 
    knows: Set[(Employee.Id, Task.Id)])

This implementation is an example of a _flat_ data model. The key feature of these kinds of models are that the different entities (employees, departments and tasks, in this case) refer to each other by using _keys_. This is a possible instance of the organization data model:

In [ ]:
val org: Organization = Organization(
    Map(
        "Product"  -> Department("Product"),
        "Quality"  -> Department("Quality"),
        "Research" -> Department("Research"),
        "Sales"    -> Department("Sales")),
    
    Map("build"    -> Task("build", 3), 
        "abstract" -> Task("abstract", 5), 
        "design"   -> Task("design", 2),
        "call"     -> Task("call", 1),
        "program"  -> Task("program", 3)),
    
    Map("Alex"     -> Employee("Alex", "Product"), 
        "Bert"     -> Employee("Bert", "Product"), 
        "Cora"     -> Employee("Cora", "Research"), 
        "Drew"     -> Employee("Drew", "Research"), 
        "Edna"     -> Employee("Edna", "Research"), 
        "Fred"     -> Employee("Fred", "Sales")),
    
    Set(
        ("Alex", "build"),
        ("Bert", "build"),
        ("Cora", "abstract"),
        ("Cora", "build"),
        ("Cora", "design"),
        ("Drew", "abstract"),
        ("Drew", "design"),
        ("Edna", "abstract"),
        ("Edna", "call"),
        ("Edna", "design"),
        ("Fred", "call")))

Flat data models are actually very close to the common _relational_ data models used in SQL persistent stores. This is the equivalent relational model of the organization database: 

![](images/relational-model.png)

According to this mapping: 
- The `Organization` class represents the whole relational _database_.
- Members of this class correspond to the different _tables_ of the database, represented as `Map`s or simple `Set`s. We have four tables: the table of departments, employees, tasks, and a table which stores which tasks employees can perform.
- The key type of `Map` can be understood as the primary key of the relational table. The value type specifies the columns of the table. By convention, the identifier type is defined by the `Id` type alias in the companion object of the value type. For instance, the `employees` table is indexed by the employee identifier (a string value), and stores the department to which the employee belongs to.
- If the primary key consists of several keys, as in the `knows` table, we use tuples. 
- If the table just consists of the key (simple or composed) we use `Set` instead of `Map` (as the `knows` table also illustrates).



### Basic queries

Complex queries typically builds upon basic queries which are directly related to the structure of the data model. In particular, they are identified from the primary key and foreign-key relations in the relational model. In the organizational database we can identify the following queries:

In [ ]:

object BasicQueries{

    // Queries from single primary keys
    
    def departments(org: Organization): Set[Department.Id] = 
        org.departments.keys.toSet

    def employees(org: Organization): Set[Employee.Id] = 
        org.employees.keys.toSet

    def tasks(org: Organization): Set[Task.Id] = 
        org.tasks.keys.toSet

    // Queries from foreign-keys
    
    def employees(dpt: Department.Id)(org: Organization): Set[Employee.Id] = 
        org.employees.filter(_._2.dpt == dpt).map(_._1).toSet
    
    // Queries from compound primary keys
    
    def performers(tsk: Task.Id)(org: Organization): Set[Employee.Id] = 
        org.knows.filter(_._2 == tsk).map(_._1)

    def capabilities(emp: Employee.Id)(org: Organization): Set[Task.Id] = 
        org.knows.filter(_._1 == emp).map(_._2)
}

import BasicQueries._

### Sample queries

__Which are the tasks of the organization which can't be performed by any employee?__

In [ ]:
class TestImpossibleTasks(
    impossibleTasks: Organization => Set[Task.Id]
) extends FlatSpec with Matchers{
    
    "impossibleTasks" should "work" in {
        impossibleTasks(org) shouldBe 
            Set("program")
    }
}

This is a conventional imperative implementation, using mutable variables:

In [ ]:
def impossibleTasks(org: Organization): Set[Task.Id] = {
    var impTasks: Set[Task.Id] = tasks(org)
    for (entry <- org.knows)
        impTasks = impTasks - entry._2
    impTasks
}

In [ ]:
run(new TestImpossibleTasks(impossibleTasks))

This works but it is not the _functional_ style. The following version is closer to what we are looking for:

In [ ]:
def impossibleTasks(org: Organization): Set[Task.Id] =
    org.knows.foldLeft(tasks(org))(
        (impTasks, entry) => impTasks - entry._2
    )

or with pattern matching syntax:

In [ ]:
def impossibleTasks(org: Organization): Set[Task.Id] =
    org.knows.foldLeft(tasks(org)){
        case (impTasks, (_, task)) => impTasks - task
    }

In [ ]:
run(new TestImpossibleTasks(impossibleTasks))

But we can do it even better. We will endorse the following implementation that uses hight-level set operations (`diff`) and HOFs (`map`):

In [ ]:
def impossibleTasks(org: Organization): Set[Task.Id] = {
    val possibleTasks = org.knows.map(entry => entry._2)
    tasks(org) diff possibleTasks
}

In [ ]:
run(new TestImpossibleTasks(impossibleTasks))

Arguably, this implementation conveys the intent of the function more clearly. It's more _declarative_. Moreover, it is more reliable since it builds upon standard methods of the Scala library (`diff` and `map`). It's true that the imperative version is also easy to read, but this is only because this is such a very simple function. We will see later on more complex examples where the functional solution shines brighter. 

__Which tasks can be performed by the employees of a given department?__

In [ ]:
class TestAllTasks(
    allTasks: Department.Id => Organization => Set[Task.Id]
) extends FlatSpec with Matchers{
    
    "allTasks" should "work" in {
        allTasks("Product")(org) shouldBe 
            Set("build")
        allTasks("Quality")(org) shouldBe 
            Set()
        allTasks("Sales")(org) shouldBe 
            Set("call")
        allTasks("Research")(org) shouldBe 
            Set("abstract", "build", "design", "call")
    }
}

The basic queries of the data model allow us to obtain all the employees of an organization, and the tasks that they can perform. So, this a first step towards the solution:

In [ ]:
// def allTasks(dpt: Department.Id)(org: Organization): Set[Task.Id]
def allTasks(dpt: Department.Id)(org: Organization): Set[Set[Task.Id]] = 
    employees(dpt)(org) map (
        emp => capabilities(emp)(org)
    )

However, this is not the signature that we need to implement, since we are returning a set of sets of tasks, not a set of tasks. In order to do it right we need also to _flatten_ the result, i.e. concatenate all the individual sets of tasks for each employee. In sum, we need the `flatMap` HOF:

In [ ]:
def allTasks(dpt: Department.Id)(org: Organization): Set[Task.Id] = 
    employees(dpt)(org) flatMap (
        emp => capabilities(emp)(org)
    )

In [ ]:
run(new TestAllTasks(allTasks))

__Which are the departments whose employees, as a team, know how to perform a given set of tasks?__

In [ ]:
class TestDptsThatKnowHowTo(
    dptsThatKnowHowTo: Set[Task.Id] => Organization => Set[Department.Id]
) extends FlatSpec with Matchers{
    
    "dptsThatKnowHowTo" should "work" in {
        dptsThatKnowHowTo(Set())(org) shouldBe 
            Set("Sales", "Product", "Quality", "Research")
        dptsThatKnowHowTo(Set("call"))(org) shouldBe 
            Set("Sales", "Research")
        dptsThatKnowHowTo(Set("call", "abstract"))(org) shouldBe 
            Set("Research")
    }
}

We can build upon the previous function `allTasks`:

In [ ]:
def dptsThatKnowHowTo(tasks: Set[Task.Id])(org: Organization): Set[Department.Id] = 
    departments(org) filter {
        dpt: Department.Id => tasks subsetOf allTasks(dpt)(org)
    }

In [ ]:
run(new TestDptsThatKnowHowTo(dptsThatKnowHowTo))

__Obtain a list of employees sorted by the number of tasks that they can perform__

In [ ]:
class TestSortedEmployees(
    sortedEmployees: Organization => List[(Employee.Id, Int)]
) extends FlatSpec with Matchers{
    
    "sortedEmployees" should "work" in {
        sortedEmployees(org) shouldBe 
            List(
              ("Alex", 1),
              ("Fred", 1),
              ("Bert", 1),
              ("Drew", 2),
              ("Cora", 3),
              ("Edna", 3))
    }
}

In [ ]:
def sortedEmployees(org: Organization): List[(Employee.Id, Int)] = 
    org.knows
        .groupBy(_._1)     // Map[Employee.Id, List[Task.Id]]
        .mapValues(_.size) // Map[Employee.Id, Int]
        .toList            // List[(Employee.Id, Int)]
        .sortBy(_._2) 

In [ ]:
run(new TestSortedEmployees(sortedEmployees))

__Which are the departments whose employees are all able to perform a given task?__

In [ ]:
class TestExpertDepsIn(
    expertDpts: Task.Id => Organization => Set[Department.Id]
) extends FlatSpec with Matchers{
    
    "expertDpts" should "work" in {
        expertDpts("abstract")(org) shouldBe 
            Set("Quality", "Research")
    }
}

The conventional imperative solution is quite complex: 

In [ ]:
def expertDepsIn(task: Task.Id)(org: Organization): Set[Department.Id] = {
    var out: Set[Department.Id] = Set()
    for (dep <- departments(org)){
        var employeesThatCantPerform: Set[Employee.Id] = 
            employees(dep)(org)
        for (emp <- employees(dep)(org)){
            for (someTask <- capabilities(emp)(org))
                if (someTask == task)
                    employeesThatCantPerform -= emp
        }
        if (employeesThatCantPerform.isEmpty) 
            out += dep
    }
    out
}

In [ ]:
run(new TestExpertDepsIn(expertDepsIn))

This is not only more complex to understand, but prone to error. In order to obtain a simpler (and functional) solution by first declaring in plain natural language the intended query:

In [ ]:
def expertDepsIn(tsk: Task.Id)(org: Organization): Set[Department.Id] = 
    // From all the departments of the organization, choose
    // those that for all its employees
    // the specified task is included in their capabilities
    ???

Then, we can formalize the natural language specification by relying on standard HOFs (`filter`, `forall`) and collection operations (`contains`):

In [ ]:
def expertDepsIn(tsk: Task.Id)(org: Organization): Set[Department.Id] = 
    departments(org).filter(
        dpt => employees(dpt)(org).forall(
            emp => capabilities(emp)(org).contains(tsk)
        )
    )

In [ ]:
run(new TestExpertDepsIn(expertDepsIn))